# Data Engineering 1: Graded Lab 01
---------------

#### Grading
For this graded lab you can get a total of 10 points. These 10 points count 10% of your final grade for the course.

#### Start
Start of the Graded Lab 01 is **Thursday, March 21st at 23:55**.

#### Deadline
Deadline for the submission of the Graded Lab 01 is **Thursday, April 11th at 23:59**.

#### Note
Check each result carefully. Use data filter, cleaning, and transformation methods wherever needed. The data can sometimes be really messy and have hidden issues.

#### Submission
You are allowed to submit the solution in groups of **two or three** students.
Submit your GradedLab01.ipynb file renamed to FirstnameStudent01LastnameStudent01_FirstnameStudent02LastnameStudent02_FirstnameStudent03LastnameStudent03.ipynb in moodle.   
Please submit a runnable python jupyter notebook file.
All other submissions will be rejected and graded with 0 points.

##### Task 01: Structured Data [4 points].    
The imdb.csv file contains a dataset extracted from IMDB with several attributes. For example, the title, plot, and the language of the movies. Read the imdb.csv file in a pandas dataframe and try to answer the following questions.

##### __(a)  What is the min, max, and the average value of the attribute 'rating' for all movies? [0.5 points].__ 

In [1]:
import pandas as pd

imdb = pd.read_csv('../Grundlagen/DE1_GradedLab01/imdb.csv')

print(f"Lowest rating: {imdb['rating'][imdb['kind'] == 'movie'].min()}")
print(f"Highest rating: {imdb['rating'][imdb['kind'] == 'movie'].max()}")
print(f"Average rating: {imdb['rating'][imdb['kind'] == 'movie'].mean():.2f}")

Lowest rating: 8.1
Highest rating: 9.3
Average rating: 8.35


##### __(b) What is the min, max, and the average value of the attribute 'cumulative worldwide gross' for all movies? [0.5 points].__ 

In [2]:
# Data cleansing
imdb['cumulative worldwide gross'] = imdb['cumulative worldwide gross'].str.replace(r'[\$,]', '', regex=True)
imdb['cumulative worldwide gross'] = imdb['cumulative worldwide gross'].str.split().str[0]
imdb['cumulative worldwide gross'] = imdb['cumulative worldwide gross'].str.replace('000000000', '0', regex=True)
imdb['cumulative worldwide gross'] = pd.to_numeric(imdb['cumulative worldwide gross'], errors='coerce')

In [3]:
# Queries
print(f"Lowest cumulative worldwide gross: {imdb['cumulative worldwide gross'][imdb['kind'] == 'movie'].min()}")
print(f"Highest cumulative worldwide gross: {imdb['cumulative worldwide gross'][imdb['kind'] == 'movie'].max()}")
print(f"Average cumulative worldwide gross: {imdb['cumulative worldwide gross'][imdb['kind'] == 'movie'].mean():.2f}")

Lowest cumulative worldwide gross: 6540000
Highest cumulative worldwide gross: 2797800564
Average cumulative worldwide gross: 390489624.68


##### __(c) What is the min, max, and the average value of the attribute 'cumulative worldwide gross' grouped by genre of the movies? [1 point].__ 

In [4]:
# Convert values of this column to lists
imdb['genres'] = imdb['genres'].apply(eval)

In [5]:
# Get all unique genres
genres = []
for cell in imdb['genres']:
    for element in cell:
        genres.append(element)

genres = list(set(genres))
genres.sort()
print(genres)



['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']


##### __(d) What is the profit (defined as cumulative worldwide gross - budget) of each movie? [1 point].__ 

In [6]:
# Data cleansing
imdb['budget'] = imdb['budget'].str.replace(r'[\$,]', '', regex=True)
imdb['budget'] = imdb['budget'].str.split().str[0]
imdb['budget'] = imdb['budget'].str.replace('000000000', '0', regex=True)
imdb['budget'] = pd.to_numeric(imdb['budget'], errors='coerce')

In [7]:
# Queries
imdb['net profit'] = imdb['cumulative worldwide gross'] - imdb['budget']
imdb[['title', 'net profit']][imdb['kind'] == 'movie']

,title,net profit
0,The Shawshank Redemption,33500000
1,The Godfather,239066411
2,The Dark Knight,819558444
3,Schindler's List,199000000
4,Pulp Fiction,205928762
...,...,...
92,Pirates of the Caribbean: The Curse of the Bla...,514264015
93,Aladdin,476050219
94,Beauty and the Beast,399967620
95,The Help,191639112


##### __(e)  What is the min, max, and the average value of the atttribute 'opening weekend united states' per month? [1 point].__ 

In [121]:
# Data cleansing
imdb['opening weekend united states'] = imdb['opening weekend united states'].str.replace(r'[\$,]', '', regex=True)
imdb['opening weekend date united states'] = imdb['opening weekend united states']
imdb['opening weekend united states gross'] = imdb['opening weekend united states'].str.split().str[0].astype(int)
imdb['opening weekend date united states'] = imdb['opening weekend date united states'].str.split().str[1:4]

In [ ]:
# Create dataframe with month and revenue for opening weekends
imdb_month_revenue = pd.DataFrame()
imdb_month_revenue['month'] = imdb['opening weekend date united states'][imdb['kind'] == 'movie'].apply(lambda x: x[1])
imdb_month_revenue['revenue'] = imdb['opening weekend united states gross'][imdb['kind'] == 'movie'].astype(int)

In [141]:
# Queries
min_revenue = imdb_month_revenue.groupby(['month']).min().astype(int).sort_values('revenue', ascending=False)
max_revenue = imdb_month_revenue.groupby(['month']).max().astype(int).sort_values('revenue', ascending=False)
average_revenue = imdb_month_revenue.groupby(['month']).mean().astype(int).sort_values('revenue', ascending=False)

In [142]:
# Merge and name the columns after the previous DataFrame
revenue_month = pd.merge(min_revenue, max_revenue, on='month', how='outer', suffixes=('_min', '_max'))
revenue_month = pd.merge(revenue_month, average_revenue, on='month', how='outer')
revenue_month = revenue_month.rename(columns={'revenue': 'revenue_avg'})
revenue_month


,revenue_min,revenue_max,revenue_avg
month,,,
May,34819017,68108790,51463903
Feb,13766814,41062440,30245305
Aug,645363,26681262,17097305
Jun,340456,110307189,39295741
Mar,235488,88411916,23604512
Sep,212440,20817053,7305612
Dec,179953,35363376,12879034
Jul,137301,160887295,41997361
Nov,99761,70467623,18304484


#### Task 02: Unstructured Data [6 points]. 
This time use the files imdb.csv from Task 01 and the imdb2.csv file. In this task we mainly use the content of the attribute 'plot' for the implementation of a retrieval system.

##### __(a)  Read the files imdb.csv and imdb2.csv in a dataframe called imdb. Add a column which is called 'plotterms' to each item. The new column should contain all terms (lower-case and cleaned from special characters) from the plot attribute which are not contained in the stopwords_english.txt file. [0.5 points]__ 

In [12]:
import pandas as pd

data = pd.read_csv('imdb2.csv')

data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'imdb2.csv'

##### __(b)  Create an inverted index for the terms in the 'plotterms' column. Use a datastructure of your choice for the implementation [0.5 points]__ 

##### __(c) Take the inverted index and create the posting list for the query 'american'. Return the posting list as well as the top5 items scored by the rating attribute. [0.5 points]__ 

##### __(d) Take the inverted index and create the posting lists for the query terms 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot). Your merging algorithm should be efficient and reduce the number of comparison to a minimum. Don't use existing python methods (like intersect or in) for the merge algorithm. Return the merged posting list as well as the top5 items scored by the rating attribute. [1 point]__ 

##### __(e) Extend your index to be able to rank the resulting items from Task 2d by the occurrence of search terms in the plotterms. Execute the query 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot) again and print the resulting items.  [1 point]__

##### __(f) Use the TF-IDF score and the cosine similarity to execute the query 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot) against all plots in the collection. Print the results with the corresponding ranking score. Also execute the query 'american' or 'dream' (the term 'american' or the term 'dream' should be contained in the plot).  [1.5 points]__